In [1]:
import xml.etree.cElementTree as et

In [29]:
import requests
import xml

def search(item):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={item}&retmode=xml&retmax=1000"
    r = requests.get(url)
    doc = xml.dom.minidom.parseString(r.text)
    id_list = []
    ids = doc.getElementsByTagName("Id")
    for i in range(len(ids)):
        id_list.append(ids[i].childNodes[0].data) 
    return id_list   


ids1_ = search("Alzheimers+AND+2019[pdat]")
ids2_ = search("Cancer+AND+2019[pdat]")


In [30]:
set(ids1)&set(ids2)

{'32322464', '32501203'}

In [2]:
file1 = et.parse('Alzheimers.xml')

In [3]:
root1 = file1.getroot()

In [4]:
ids1 = []
for i in root1[3]:
    ids1.append(i.text)

In [5]:
file2 = et.parse('Cancer.xml')
root2 = file2.getroot()
ids2 = []
for i in root2[3]:
    ids2.append(i.text)


In [6]:
len(ids1)==len(ids2)

True

In [7]:
set(ids1)&set(ids2)

{'32322464', '32501203'}

In [8]:
import requests
import xml.dom.minidom as m
import time
def get_info(pmid):
    try:
        r = requests.get(f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id={pmid}" )
        time.sleep(1)
    except:
        print(f'get {pmid} failed')
        
    doc = m.parseString(r.text)
    abstract, mesh = '', []
    titles = doc.getElementsByTagName("Title") 
    if not titles:
        titles = doc.getElementsByTagName('BookTitle')
    abstracts = doc.getElementsByTagName("AbstractText") 
    meshes = doc.getElementsByTagName("DescriptorName") 

    if  not abstracts:
        abstract = ''
    else:
        for abst in abstracts:     
#             abstract.setdefault(abst.getAttribute('Label'), '')
#             for node in abst.childNodes:
            tree = et.fromstring(abst.toxml())
            abstract+= et.tostring(tree, method='text').decode()
            abstract+='\n'
    if not meshes:
        mesh = meshes
    else:
        for mes in meshes:
            for node in mes.childNodes:               
                mesh.append(node.data) 


                    
    title = titles[0].childNodes[0].wholeText


   
    return title, abstract, mesh

In [105]:
import json
j ={}
for pmid in ids1+ids2:
    j[pmid] = {}
#     print(pmid)
    title, abstract, mesh = get_info(pmid)
    if pmid in ids1 and pmid in ids2:
        
        
        query = 'Alzheimer&Cancer'
    elif pmid in ids1:
        query = 'Alzheimer'
    else:
        query = 'Cancer'
    j[pmid]['ArticleTitle'], j[pmid]['AbstractText'], j[pmid]['query'], j[pmid]['mesh'] = title, abstract, query, mesh

with open('articles2019_n.json', 'w', encoding='utf8') as f:
    json.dump(j, f)
    
    

In [10]:
# j_c={}

# for pmid in ids2:
#     j_c[pmid] = {}
#     title, abstract, mesh = get_info(pmid)
#     query = 'Cancer'
#     j_c[pmid]['ArticleTitle'], j_c[pmid]['AbstractText'], j_c[pmid]['query'], j_c[pmid]['mesh'] = title, abstract, query, mesh

# with open('Cancer.json', 'w', encoding='utf8') as f:
#     json.dump(j_c, f)
    
    

In [17]:
# j_a={}

# for pmid in ids1:
#     j_a[pmid] = {}
#     title, abstract, mesh = get_info(pmid)
#     query = 'Alzheimer'
#     j_a[pmid]['ArticleTitle'], j_a[pmid]['AbstractText'], j_a[pmid]['query'], j_a[pmid]['mesh'] = title, abstract, query, mesh

# with open('Cancer.json', 'w', encoding='utf8') as f:
#     json.dump(j_c, f)

In [18]:
# j = {}
# for i in j_a:
#     j[i] = j_a[i]
# for i in j_c:
#     j[i] = j_c[i]


In [20]:
j['32322464']['query'] = 'Alzheimer&Cancer'
j['32501203']['query']  = 'Alzheimer&Cancer'

In [21]:
import json
with open('articles2019_n1.json', 'w', encoding='utf8') as f:
    json.dump(j,f)